# Circuit Topology analysis of Single-Cell HiC pairs of contact lists

Using Barbara's ExtractTopology_singlecellsHiC.ipynb we will generate topology parameters and images

In [10]:
# Load all packages
import numpy as np
import os
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline
import PIL.Image
from read_HiC import name_chromosomes

# Import genome_topology

from plotting_tools import set_layout
from genome_topology import normalize_psc
from genome_topology import get_matrix
from genome_topology import fractal_dimension
from genome_topology import make_graph

## Calculate topological parameters looping over all chromosomes in a cell

In [27]:
# Running for the CT_parameters
def process_file(file,
                 cell,
                 path_savematrix,
                 path_savedata,
                 n_all_chr=24,
                 save_data=True,
                 save_matrix=True,
                 exception_files=[]):  # Initialize the list of exception files
    n_all_chr+=1
    csv_path = os.path.join(path_savedata, f"Top_parameters_{cell}.csv")
    if not os.path.isfile(csv_path):
        if file.endswith('.txt'):
            contacts = pd.read_csv(file, sep='\t')
            chr_vec = name_chromosomes(n_all_chr)

            # Check if the directory for saving matrices exists, create it if not
            if not os.path.exists(path_savematrix):
                os.makedirs(path_savematrix)

            P = np.zeros(n_all_chr)
            S = np.zeros(n_all_chr)
            X = np.zeros(n_all_chr)
            Dim_fractal = np.zeros(n_all_chr)
            clustering = np.zeros(n_all_chr)
            r2_fractalfit = np.zeros(n_all_chr)
            N_contacts = np.zeros(n_all_chr)

            for t, chrom in enumerate(chr_vec):
                
                contacts_chr = contacts[(contacts['#chrA'] == chrom) & (contacts['chrB'] == chrom)]
                index = [contacts_chr['pos_A'], contacts_chr['pos_B']]
                index = np.array(index)
                index = np.transpose(index)
                N_contacts[t] = len(index)

                mat, psc = get_matrix(index, chrom)

                # Check if mat is empty or zero-size
                if mat.size == 0:
                    print(f"Skipping {chrom} from cell {cell} due to empty matrix.")
                    continue

                # Normalize image data

                P[t], S[t], X[t] = normalize_psc(psc, N_contacts[t])

                try:
                    Dim_fractal[t], r2_fractalfit[t] = fractal_dimension(mat, plot_fig=0)
                except OverflowError:
                    # Handle the overflow error gracefully
                    Dim_fractal[t], r2_fractalfit[t] = np.nan, np.nan

                try:                  
                    if 'pairs_of_contacts' in contacts_chr:
                        
                        G = nx.Graph()
                        for [a,b],w in zip(index, contacts_chr['pairs_of_contacts'].tolist()):
                            G.add_edge(a, b, weight=w)
                        clustering[t] = nx.average_clustering(G, weight = "weight") #In case of coarse graining use weights as pairs_of_contacts values
                        
                    else:
                        G = make_graph(index)
                        clustering[t] = nx.average_clustering(G)
                except ZeroDivisionError as err:
                    pass

                if save_matrix:
                    # Save the numpy file
                    npy_path_mat = os.path.join(path_savematrix, f"top_matrix_{cell}_{chrom}_mat.npy")
                    npy_path_index = os.path.join(path_savematrix, f"top_matrix_{cell}_{chrom}_index.npy")
                    try:
                        np.save(npy_path_mat, mat)  
                        np.save(npy_path_index, index)  
                        print("Saved numpy file to:", npy_path_mat)
                    except Exception as e:
                        print("Error saving numpy file:", e)
                        exception_files.append(npy_path)  # Add the file to the list of exception files

            # Save results
            if save_data:
                topology_parameters = {'Parallel (%)': P, 'Series (%)': S, 'Cross (%)': X,
                                       'N contacts': N_contacts, 'Fractal dimension': Dim_fractal,
                                       'r squared': r2_fractalfit, 'Clustering': clustering}

                topology_parameters = pd.DataFrame(topology_parameters)
                csv_path = os.path.join(path_savedata, f"Top_parameters_{cell}.csv")
                try:
                    topology_parameters.to_csv(csv_path)
                    print("Saved data to:", csv_path)
                except Exception as e:
                    print("Error saving data:", e)
                    exception_files.append(csv_path)  # Add the file to the list of exception files

In [28]:
#Defining variables
cell_types = ['Macrophage','Macrophage-Mtb','Monocyte']
discretisation_steps = ["10e3", "5e4", "10e4", "5e5", "10e5"]

In [29]:
#Running the generation of Circuit Topology files
exception_files = [] 

for step in discretisation_steps:
    for cell_type in cell_types:
        directory = f'/media/msbb/ssd2/Yasmine_copy/Sci_pairs_resolutions/Sci_pairs_all/{step}/{cell_type}_Sci_pairs'
        files = os.listdir(directory)

        os.makedirs(f'/media/msbb/ssd2/Yasmine_copy/results_Sci/Resolutions_v2/{step}/{cell_type}/matrices/',exist_ok=True)
        os.makedirs(f'/media/msbb/ssd2/Yasmine_copy/results_Sci/Resolutions_v2/{step}/{cell_type}/CT_parameters/',exist_ok=True)
        files = os.listdir(directory)
        files_paths = [os.path.join(directory, file) for file in files if file.endswith('.txt')]
       
                
        for file,file_path in zip(files,files_paths): 
            cell = file[:-4]
            process_file(cell=cell,
                         file=file_path,
                         n_all_chr=22,
                         save_data=False,
                         save_matrix=True,
                         path_savematrix=f'/media/msbb/ssd2/Yasmine_copy/results_Sci/Resolutions_v2/{step}/{cell_type}/matrices/{cell}',
                         path_savedata=f'/media/msbb/ssd2/Yasmine_copy/results_Sci/Resolutions_v2/{step}/{cell_type}/CT_parameters/',
                         exception_files=exception_files)

Saved numpy file to: /media/msbb/ssd2/Yasmine_copy/results_Sci/Resolutions_v2/10e3/Macrophage/matrices/V300067419_L03_60.cell.TCTAACGG.sorted.pairs_all.pairs/top_matrix_V300067419_L03_60.cell.TCTAACGG.sorted.pairs_all.pairs_chr1_mat.npy
Saved numpy file to: /media/msbb/ssd2/Yasmine_copy/results_Sci/Resolutions_v2/10e3/Macrophage/matrices/V300067419_L03_60.cell.TCTAACGG.sorted.pairs_all.pairs/top_matrix_V300067419_L03_60.cell.TCTAACGG.sorted.pairs_all.pairs_chr2_mat.npy
Saved numpy file to: /media/msbb/ssd2/Yasmine_copy/results_Sci/Resolutions_v2/10e3/Macrophage/matrices/V300067419_L03_60.cell.TCTAACGG.sorted.pairs_all.pairs/top_matrix_V300067419_L03_60.cell.TCTAACGG.sorted.pairs_all.pairs_chr3_mat.npy
Saved numpy file to: /media/msbb/ssd2/Yasmine_copy/results_Sci/Resolutions_v2/10e3/Macrophage/matrices/V300067419_L03_60.cell.TCTAACGG.sorted.pairs_all.pairs/top_matrix_V300067419_L03_60.cell.TCTAACGG.sorted.pairs_all.pairs_chr4_mat.npy
Saved numpy file to: /media/msbb/ssd2/Yasmine_copy/r

Traceback (most recent call last):
  File "/home/msbb/anaconda3/envs/sci_Hi-C/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_25443/2796190092.py", line 16, in <module>
    process_file(cell=cell,
  File "/tmp/ipykernel_25443/1616778630.py", line 37, in process_file
    mat, psc = get_matrix(index, chrom)
               ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/media/msbb/ssd2/Yasmine_copy/genome_topology.py", line -1, in get_matrix
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/msbb/anaconda3/envs/sci_Hi-C/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 2144, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/msbb/anaconda3/envs/sci_Hi-C/lib/python3.11/site-packages/IPython/core/ultratb.py